In [2]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

no background image


In [3]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironment(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 205.37it/s]


In [4]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, n, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0119_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.9)
world_trajGen.ShowRewards(title='真实奖励')

In [5]:
world_trajGen.features[list(world_trajGen.features.keys())[1]]

array([[0.68842549, 0.82370043, 0.8446259 , 0.7702021 , 0.60654472,
        0.38787123, 0.23613305, 0.1562775 , 0.10448166, 0.06774722],
       [0.82042785, 1.        , 0.99240456, 0.83546335, 0.62715772,
        0.39778139, 0.24606274, 0.16319477, 0.10924787, 0.07102722],
       [0.81600209, 0.9555617 , 0.9026075 , 0.71372825, 0.51101263,
        0.34447171, 0.23189477, 0.15860034, 0.10730314, 0.06996647],
       [0.73744249, 0.78036497, 0.71528291, 0.57798141, 0.41456305,
        0.28898077, 0.20531095, 0.1450917 , 0.09943983, 0.0648924 ],
       [0.58286942, 0.56711416, 0.50625448, 0.43057221, 0.3229855 ,
        0.23583086, 0.1732946 , 0.12511483, 0.08678237, 0.05661016],
       [0.37670883, 0.36583894, 0.3369425 , 0.29518244, 0.23873234,
        0.18569899, 0.14032202, 0.10295969, 0.07178714, 0.04621506],
       [0.22997896, 0.2381865 , 0.22790205, 0.20591308, 0.17503143,
        0.1416795 , 0.10948763, 0.0808343 , 0.05595808, 0.03470749],
       [0.15387379, 0.16169192, 0.1588619

In [6]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=20)

100%|██████████| 500/500 [00:00<00:00, 4775.82it/s]


,m,trajs
0,1,"[(5, 1, 15), (15, 1, 25), (25, 1, 35), (35, 1,..."
1,2,"[(1, 4, 2), (2, 4, 3), (3, 4, 3), (3, 4, 4), (..."
2,3,"[(1, 4, 0), (0, 4, 10), (10, 1, 20), (20, 1, 2..."
3,4,"[(0, 4, 0), (0, 4, 0), (0, 4, 1), (1, 4, 2), (..."
4,5,"[(2, 4, 3), (3, 4, 4), (4, 4, 5), (5, 1, 15), ..."
...,...,...
495,496,"[(0, 4, 0), (0, 4, 1), (1, 4, 1), (1, 4, 1), (..."
496,497,"[(9, 0, 9), (9, 0, 9), (9, 0, 9), (9, 0, 9), (..."
497,498,"[(6, 1, 16), (16, 1, 26), (26, 3, 26), (26, 3,..."
498,499,"[(8, 3, 7), (7, 3, 7), (7, 3, 6), (6, 1, 6), (..."


In [7]:
world = GridWorld(
    states_features=world_trajGen.states_features,
    expert_trajs=world_trajGen.df_trajs,
    width=10,
    height=10,
    active_all=False,
    trans_prob=0.6,
    real_reward_mat = world_trajGen.real_rewards_matrix
)
world.ShowGridWorld_Count_log(title='轨迹计数分布(对数)')

In [8]:
world.real_reward_arr.shape

(85,)

In [9]:
world.ShowGridWorld_Activated()

In [27]:
dme = DMEIRL(world,layers = (16,16),lr = 0.0005,log='ttttt2',log_dir='test_run')
rewards_result1,rewards1 = dme.train(n_epochs=1000,save = False,demo=True)

 98%|█████████▊| 979/1000 [32:13<00:43,  2.08s/it]

In [28]:
world.ShowRewardsResult(rewards_result1,title='DMEIRL奖励复原结果')

In [29]:
world.ShowRewardsAnimation(rewards1,title='DMEIRL奖励复原动画')

In [1]:
dme.SaveModel('1_model.pth')

NameError: name 'dme' is not defined